# Conexiones módulo PWM UC3844
[Datasheet UC3844](../Datasheets/PWM_UC3844.pdf)

## Frecuencia
Cálculo de $C_T$ y $R_T$ para fijar $f_{sw}$

$$f_{sw} = \frac{1.72}{C_T \cdot R_T}$$

Queremos $f_{sw} = 150 kHz$

In [16]:
from utils import C_values, R_values, to_unit, parallel

fsw = 150E3
Rd = 0

tol = 0.01
CT = []
RT = []

for Ct in C_values:
    if 1E-9 < Ct < 100E-9:
        for Rt in R_values:
            if 5E3 < Rt < 100E3: 
                f = 1.72/(Ct*Rt)
                if fsw*(1-tol) < f < fsw*(1+tol):
                    CT.append(Ct)
                    RT.append(Rt)
        
print(f'Se encontraron {len(CT)} opciones')
for i in range(len(CT)):
    print('CT = ' + to_unit(CT[i]) + 'F con RT = ' + to_unit(RT[i]) + 'Ohm, resultando en fsw = ' + to_unit(1.72/(CT[i]*RT[i]), 5) + 'Hz')

if len(CT) == 0:
    print('Buscando con resistencias en paralelo\n')

    Rpar = []
    for Ct in C_values:
        if 1E-9 < Ct < 200E-9:
            i = 0
            j = 0
            for i in range(len(R_values)):
                for j in range(i, len(R_values)):
                    Rt = parallel(R_values[i], R_values[j])
                    if 5E3 < Rt < 100E3: 
                        f = 1.72/(Ct*Rt)
                        if fsw*(1-tol) < f < fsw*(1+tol):
                            CT.append(Ct)
                            RT.append((R_values[i], R_values[j]))

    print(f'Se encontraron {len(CT)} opciones')
    for i in range(len(CT)):
        print('CT = ' + to_unit(CT[i]) + 'F con RT1 = ' + to_unit(RT[i][0]) + 'Ohm y RT2 = ' + to_unit(RT[i][1]) + 'Ohm => RT = ' + to_unit(parallel(RT[i][0], RT[i][1])) + 'Ohm, resultando en fsw = ' + to_unit(1.72/(CT[i]*parallel(RT[i][0], RT[i][1])), 5) + 'Hz')
        print('Rango: RT1 = 5.6KOhm -> ' + 'fsw = ' + to_unit(1.72/(CT[i]*parallel(5600, RT[i][1])), 5) + 'Hz | RT1 = 10.6kOhm -> ' + 'fsw = ' + to_unit(1.72/(CT[i]*parallel(10.6E3, RT[i][1])), 5) + 'Hz\n')

Se encontraron 0 opciones
Buscando con resistencias en paralelo

Se encontraron 7 opciones
CT = 1.5nF con RT1 = 8.2kOhm y RT2 = 100kOhm => RT = 7.58kOhm, resultando en fsw = 151.3kHz
Rango: RT1 = 5.6KOhm -> fsw = 216.23kHz | RT1 = 10.6kOhm -> fsw = 119.64kHz

CT = 1.5nF con RT1 = 8.2kOhm y RT2 = 120kOhm => RT = 7.68kOhm, resultando en fsw = 149.39kHz
Rango: RT1 = 5.6KOhm -> fsw = 214.32kHz | RT1 = 10.6kOhm -> fsw = 117.73kHz

CT = 1.5nF con RT1 = 10kOhm y RT2 = 33kOhm => RT = 7.67kOhm, resultando en fsw = 149.41kHz
Rango: RT1 = 5.6KOhm -> fsw = 239.51kHz | RT1 = 10.6kOhm -> fsw = 142.92kHz

CT = 2nF con RT1 = 6.8kOhm y RT2 = 39kOhm => RT = 5.79kOhm, resultando en fsw = 148.52kHz
Rango: RT1 = 5.6KOhm -> fsw = 175.62kHz | RT1 = 10.6kOhm -> fsw = 103.18kHz

CT = 2.2nF con RT1 = 5.6kOhm y RT2 = 68kOhm => RT = 5.17kOhm, resultando en fsw = 151.11kHz
Rango: RT1 = 5.6KOhm -> fsw = 151.11kHz | RT1 = 10.6kOhm -> fsw = 85.254kHz

CT = 2.2nF con RT1 = 5.6kOhm y RT2 = 82kOhm => RT = 5.24kOhm, resu

Podemos elegir cualquiera entre:

CT = 1.5nF con RT1 = 8.2kOhm y RT2 = 100kOhm => RT = 7.58kOhm, resultando en fsw = 151.3kHz

	Rango: RT1 = 5.6KOhm -> fsw = 216.23kHz | RT1 = 10.6kOhm -> fsw = 119.64kHz

CT = 1.5nF con RT1 = 8.2kOhm y RT2 = 120kOhm => RT = 7.68kOhm, resultando en fsw = 149.39kHz

	Rango: RT1 = 5.6KOhm -> fsw = 214.32kHz | RT1 = 10.6kOhm -> fsw = 117.73kHz

CT = 2nF con RT1 = 6.8kOhm y RT2 = 39kOhm => RT = 5.79kOhm, resultando en fsw = 148.52kHz

	Rango: RT1 = 5.6KOhm -> fsw = 175.62kHz | RT1 = 10.6kOhm -> fsw = 103.18kHz

CT = 2.2nF con RT1 = 6.8kOhm y RT2 = 22kOhm => RT = 5.19kOhm, resultando en fsw = 150.51kHz

	Rango: RT1 = 5.6KOhm -> fsw = 175.15kHz | RT1 = 10.6kOhm -> fsw = 109.29kHz

Al final pusimos $C_T = $ y $R_T = $ + 1 PRESET de $5k \Omega$ ajustable

$\Rightarrow f_{sw} = 150kHz$

## Divisor de tensión medida
$\overline{V_o} = 16V$

$V_{comp} = 2.55V$

$V_{ref} = 5.1V$

$V_{o_{comp}} = \frac{R_a}{R_a + R_b} \cdot \overline{V_o} \approx V_{comp}$

In [1]:
from utils import R_values, to_unit

Vcomp = 2.3212
Vo = 5.1

tol = 0.002
Ra = []
Rb = []

for ra in R_values:
	if 100 < ra < 150E3: 
		for rb in R_values:
			if 100 < rb < 150E3: 
				Vo_comp = Vo*ra/(ra + rb)
				if Vcomp*(1-tol) < Vo_comp < Vcomp*(1+tol):
					Ra.append(ra)
					Rb.append(rb)
        
print(f'Se encontraron {len(Ra)} opciones')
for i in range(len(Ra)):
    print('Ra = ' + to_unit(Ra[i]) + 'Ohm con Rb = ' + to_unit(Rb[i]) + 'Ohm, resultando en Vo_comp = ' + to_unit(Vo*Ra[i]/(Ra[i] + Rb[i]), 5) + 'V')

Se encontraron 6 opciones
Ra = 150Ohm con Rb = 180Ohm, resultando en Vo_comp = 2.3182V
Ra = 1kOhm con Rb = 1.2kOhm, resultando en Vo_comp = 2.3182V
Ra = 1.5kOhm con Rb = 1.8kOhm, resultando en Vo_comp = 2.3182V
Ra = 10kOhm con Rb = 12kOhm, resultando en Vo_comp = 2.3182V
Ra = 15kOhm con Rb = 18kOhm, resultando en Vo_comp = 2.3182V
Ra = 100kOhm con Rb = 120kOhm, resultando en Vo_comp = 2.3182V


Tomamos $R_a = 5.6k\Omega$ y $R_b = 33k\Omega$

$\Rightarrow V_{o_{comp}} = 2.3212$

Tomamos $R_a = 10k\Omega$ y $R_b = 12k\Omega$

$\Rightarrow V_{{ref}_{comp}} = 2.3182V$

## Compensación por realimentación

Como no tenemos la transferencia todavía no la hicimos